<a href="https://colab.research.google.com/github/phi1z/1yanagiLab/blob/main/XRD_Diameter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XRDの結果から粒径を計算するWebUIです

## データの準備
1.   XRDの解析ソフトで測定データを読み込みこんで下さい
2.   最適化→**FP法**→**対数正規分布**を選択して実行ボタンを押してください
3.   自動検索から、元素を指定して結晶相を特定してください
4.   **ピークリストを保存**でピーク表(.csv)を保存してください

## 利用方法

1.   <font color= “blue”>**前提ファイルの読み込み**</font>を実行する
2.   ディレクトリー名(`output_dir`)を入力して実行。左のフォルダーにdataなどが作成されます(**左のフォルダーアイコン**をクリックすると、エクスプローラーが開きます。フォルダーが見当たらないときは、上の<font color= “red”>**更新ボタン**</font>を押して下さい)
3.   data/[ディレクトリー名]/に**ピーク表**(csvファイル)をアップロード。<font color= “blue”>**ファイルを確認**</font>を実行して**ファイルを選択**します

4.   ピークリストを確認して、解析する結晶相を指定
5.   結晶相のピークリストを確認して、単位(`unit_nano`)と有効数字(`sig_diameter, sd`)を設定
6.   <font color= “blue”>**解析実行**</font>で粒径が計算されます。また、自動で`out_put`に保存されます

## 結果の読み方

###出力結果

---

| mathod |	size |	sigma | 計算方法 |
| ---- | ---- | ---- | ---- |
| highest |	11.1	| 0.11 | 最も高いピークの粒径分布です |
| average	| 22.2	| 0.22 | すべてのピークの平均です |
| weight	| 33.3	| 0.33 | ピークの高さを重みとして加重平均を取ります |
| weight_area	| 44.4	| 0.44 | 積分面積を重みとして加重平均を取ります |

---

※粒径は**結晶ごとに**算出しています


In [252]:
#@title ##前提ファイルの読み込み (**とりあえず押して！**)

!pip install ipywidgets==7.7.1

from IPython.display import display
import pandas as pd
import numpy as np
import os
import ipywidgets as widgets

def round_to_sig(x, sig=3):
  return round(x, sig - int(np.floor(np.log10(abs(x)))) - 1)

def read_peaks(filename):
  peaks_raw = pd.read_csv(filename, encoding='shift_jis')

  peaks = peaks_raw[ ['2θ(deg)', '高さ(cps)', '相対高さ','相対積分強度', '結晶子サイズ(Å)', 'ESD.9', '結晶相名' ] ]
  peaks = peaks.rename(columns={'2θ(deg)':'2theta','高さ(cps)':'high', '相対高さ':'high_r', '相対積分強度':'area_r', '結晶子サイズ(Å)':'size', 'ESD.9':'size_sd','結晶相名':'name'})

  peaks['name'] = peaks['name'].str.replace('),','):')
  new_rows = []
  for _, row in peaks.iterrows():
    if ':' not in row['name']:
      new_row = row.copy()
      new_rows.append(new_row)
    else:
      materials = row['name'].split(':')
      for material in materials:
        new_row = row.copy()
        new_row['name'] = material
        new_rows.append(new_row)

  peaks = pd.DataFrame(new_rows)

  split_df = peaks['name'].str.split('(', expand=True, n=1)
  peaks['material'] = split_df[0].str.strip()
  # Check if all unique materials are 'Unknown'
  if (split_df[0].unique() == 'Unknown').all():
    peaks['hkl'] = '000' # Set 'hkl' to '000' if all materials are Unknown
  else:
    peaks['hkl'] = split_df[1].str.replace('\)', '', regex=True)
    peaks['hkl'] = peaks['hkl'].str.replace(',','')
  peaks = peaks.drop(columns=['name'])
  return peaks

def get_max(data, mode="highest"):
  if mode == "highest":
    size = data.loc[data['high_r'] == 100, 'size']
    size = size.iloc[0]
    sigma = data.loc[data['high_r'] == 100, 'size_sd']
    sigma = sigma.iloc[0]
  elif mode == "average":
    size = data['size'].mean()
    sigma = data['size_sd'].mean() / np.sqrt(len(data))
  elif mode == "weight":
    size = np.average(data['size'], weights=data['high_r'])
    sigma = np.sqrt( ((data['size_sd'] * data['high_r']) ** 2).sum() / (data['high_r']**2).sum()) / np.sqrt(len(data))
  elif mode == "weight_area":
    size = np.average(data['size'], weights=data['area_r'])
    sigma = np.sqrt( ((data['size_sd'] * data['area_r']) ** 2).sum() / (data['area_r']**2).sum()) / np.sqrt(len(data))
  else:
    raise ValueError("Invalid mode")

  return [size, sigma]

def compare_size(file, material, sig=[1,2], mode="A"):
  data = read_peaks(file)
  data_mat = data[data['material'] == material]
  df = pd.DataFrame(columns=['mathod', 'size', 'sigma'])
  for method in ['highest', 'average', 'weight', 'weight_area']:
    size, sigma = get_max(data_mat, mode=method)
    if mode == "n":
      size = size / 10
      sigma = sigma / 10
    size = round(size, sig[0])
    sigma = round_to_sig(sigma, sig[1])
    df.loc[len(df)] = [method, size, sigma]
  return df

# 出力をクリア
from IPython.display import clear_output
clear_output()



In [ ]:
#@title ディレクトリー名の決定 { display-mode: "form"}
#@markdown ##実行ボタンを押してディレクトリー名を決定


#@markdown ##ディレクトリー名を入力
dir_name = "sample" # @param {"type":"string"}

os.makedirs("data", exist_ok=True)
dir_name_ud = os.path.join("data", dir_name)
os.makedirs(dir_name_ud, exist_ok=True)

print(f"ディレクトリ名: {dir_name}")


##ファイルをアップロード

###`data/[dir_name]/`にピーク表(.csv)を<font color= “orange”>**アップロード**</font>して下さい



In [ ]:
#@title ファイルを確認 { display-mode: "form"}
#@markdown ##1. 実行ボタンを押してファイルを確認
#@markdown ##2. 解析するファイルを選択 (実行ボタンは押さない)
files = os.listdir(dir_name_ud)
files = np.sort(files)

print(f"ファイル数: {len(files)}")

# ドロップダウンリストを作成
this_file = widgets.Dropdown(
    options=files,
    value=files[0],
    description='ファイルを選択:',
    disabled=False,
    layout={'width': '50%', 'height': '40px'},
    style={'description_width': 'initial'},
)

# 表示
display(this_file)

In [ ]:
#@title ピークリストを確認

peaks = read_peaks(os.path.join(dir_name_ud, this_file.value))
display(peaks)

In [ ]:
#@title 結晶相を決定
#@markdown ##1. 実行ボタンを押して結晶相を選択
#@markdown ##2. 解析する結晶相を選択 (実行ボタンは押さない)

peaks = read_peaks(os.path.join(dir_name_ud, this_file.value))
materials = np.sort(peaks['material'].astype(str).unique())

print(f"結晶相数: {len(materials)}")
for i,material in enumerate(materials):
  print(f"{i+1}: {material}")

# ドロップダウンリストを作成
this_material = widgets.Dropdown(
    options=materials,
    value=materials[0],
    description='選択:',
    disabled=False,
    layout={'width': '50%', 'height': '40px'},
    style={'description_width': 'initial'},
)

# 表示
display(this_material)

In [ ]:
#@title 結晶相のピークリストを確認

peaks = peaks[peaks['material'] == this_material.value]
display(peaks)

In [ ]:
#@title ##オプション { display-mode: "form" , run: "auto"}

#@markdown ##1. 単位をナノメートル(nm)に

unit_nano = True # @param {"type":"boolean"}
unit = "A"
if unit_nano:
  unit = "n"
#@markdown ---
#@markdown ##2. 有効数字と桁数
#@markdown ###粒径の最小位桁
sig_diameter = 1 # @param {"type":"integer"}

#@markdown ###分散の有効数字
sig_sd = 2 # @param {"type":"integer"}

Sig = [sig_diameter, sig_sd]

unit_text = "オングストローム(Å)"
if unit == "n":
  unit_text = "ナノメートル(nm)"

print(f"単位: {unit_text}")
print(f"粒径の最小位桁: {sig_diameter}")
print(f"分散の有効数字: {sig_sd}")

In [ ]:
#@title 解析実行

table = compare_size(os.path.join(dir_name_ud, this_file.value), this_material.value, sig=Sig, mode=unit)

print("===計算条件===")
print(f"ファイル名: {this_file.value}")
print(f"結晶相: {this_material.value}")
print(f"単位: {unit_text}")
print(f"粒径の最小位桁: {sig_diameter}")
print(f"分散の有効数字: {sig_sd}\n")

print("===ファイル保存===")
os.makedirs("output", exist_ok=True)
file_name = os.path.join("output", f"Diameter_{this_file.value.replace('.csv','')}_{this_material.value}.csv")
table.to_csv(file_name, index=False)
print(f"保存先: {file_name}\n")

print("===解析結果===")
display(table)
